
# 필요한 라이브러리 설치 및 import 
1. Import and Install Dependencies

GPU를 사용하는 tensorflow와 mediapipe 설치 <br>

 mediapipe란 ? 주로 멀티 모델 오디오, 비디오 또는 모든 시계열 데이터를 구축하는 데 사용되는 프레임 워크입니다. <br>



In [1]:
# 필요한 라이브러리 설치 

#using tensorflow with GPU, mediapipe detacting movements
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [7]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MP Holistic을 사용하여 이미지의 좌표 출력하기 

#mp_holistic = mp.solutions.holistic <br>
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

#참고 사이트 <br>
#https://google.github.io/mediapipe/solutions/holistic.html



#mp_drawing = 점을 연결하여 선을 보여준다.

In [23]:
mp_holistic = mp.solutions.holistic 
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

#Live perception of simultaneous human pose, face landmarks, 
#and hand tracking in real-time on mobile devices can enable various modern life applications
#https://google.github.io/mediapipe/solutions/holistic.html

mp_drawing = mp.solutions.drawing_utils # Drawing utilities
# 점을 연결하여 선을 보여준다.

In [8]:
# 이미지의 좌표를 추적하는 함수 만들기 
#이미지 -> RGB형식으로 만들어 모델에 대입한 결과를 results에 담고, 원상태로 만든 이미지와 results를 출력한다. 

def mediapipe_detection(image, model):   # 대상을 추적한다. 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction 모델에 대입 
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


In [24]:
print(results)
#<class 'mediapipe.python.solution_base.SolutionOutputs'>

results.face_landmarks.landmark
#x: 0.5144715905189514
#y: 0.24100039899349213
#z: -0.015107447281479836

#results -> <face, pose, left_hand, right_hand> 총 네 개의 결과값이 있다,
#하나의 값 (results.face_landmarks.landmark) 당 x,y,z 총 3개의 값이 3차원으로 존재한다. 

NameError: name 'results' is not defined

# 3. 위에서 얻은 results의 값을 numpy형태로 출력하기 
Extract Keypoint Values

results.pose_landmarks.landmark -> [res.x, res.y, res.z, res.visibility]으로 총 4개의 값으로 구성 <br>
그외는 <br>
results.face_landmarks.landmark -> [res.x, res.y, res.z]으로 구성 


In [5]:
# pose, face, lh, rh의 순서로 저장한다. 
# np.concatenate 를 하여 네개의 값을 하나의 넘파이 배열로 만든다.  
#만약 frame에 값이 출력되지 않았다면, (사진에서 왼손이 보이지 않아 left_hand의 result가 none으로 나올 경우) 0으로 채운다. 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#extract_keypoints(results).shape 
# (1662,) -> 33*4 + 468*3  + 21*3 + 21*3  

# 훈련데이터 동영상 넘파이 배열로 저장하기 
Convert Keypoint Values from videos

< 데이터 출처>  <br>
: Ai hub https://aihub.or.kr/keti_data_board/language_intelligence

<데이터 구성> <br>
데이터 크기 : 3000개의 수화 동영상 <br>
녹화 참여 인원수 : 총 10명 <br>
단어 종류 : 419개 

<데이터프레임 설명> <br>
dt : AI hub에서 가져온 동영상 파일 설명 엑셀파일 원본 <br>
dt1 : 문장과 단어가 모두 포함되어 있는 dt원본파일에서 단어 동영상만 추출 <br>
list_a : dt1의 한국어 컬럼을 단어의 개수 별로 정리한 데이터 프레임 <단어/횟수>로 구성.  

## 필요한 데이터프레임 생성 

In [14]:
#tensorflow에서 엑셀파일 여는 라이브러리 다운 

#!pip install pandas
#!pip install openpyxl
#!pip install fsspec

In [3]:
#!pip install pandas
#!pip install openpyxl
#!pip install fsspec

In [31]:
#dt 데이터프레임 생성 
import pandas as pd
dt=pd.read_excel("C://Users//User//sign//10.09.xlsx")
dt.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어,번호.1
0,1.0,1.0,2017.0,정면,단어,KETI_SL_0000000001.MOV,0.0,1.0
1,2.0,1.0,2017.0,정면,단어,KETI_SL_0000000002.MOV,1.0,2.0
2,3.0,1.0,2017.0,정면,단어,KETI_SL_0000000003.MOV,2.0,3.0
3,4.0,1.0,2017.0,정면,단어,KETI_SL_0000000004.MOV,3.0,4.0
4,5.0,1.0,2017.0,정면,단어,KETI_SL_0000000005.MOV,4.0,5.0


In [4]:
#dt 데이터프레임 생성 
import pandas as pd
dt=pd.read_excel("E://Users//User//sign//10.09.xlsx")
dt.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어,번호.1
0,1.0,1.0,2017.0,정면,단어,KETI_SL_0000000001.MOV,0.0,1.0
1,2.0,1.0,2017.0,정면,단어,KETI_SL_0000000002.MOV,1.0,2.0
2,3.0,1.0,2017.0,정면,단어,KETI_SL_0000000003.MOV,2.0,3.0
3,4.0,1.0,2017.0,정면,단어,KETI_SL_0000000004.MOV,3.0,4.0
4,5.0,1.0,2017.0,정면,단어,KETI_SL_0000000005.MOV,4.0,5.0


In [9]:
for i in range(len(dt['한국어'])):
    try:
        dt['한국어'][i]=int(dt['한국어'][i])
        dt['한국어'][i]=srt(dt['한국어'][i])
    except:
        dt['한국어'][i]==dt['한국어'][i]

C:\Users\solag\AppData\Local\Temp/ipykernel_11512/3882441389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i]=int(dt['한국어'][i])


In [32]:
# dt 한국어컬럼 중 float 형 int형으로 바꾸기 

for i in range(len(dt['한국어'])):    #4704
    try:
        dt['한국어'][i] = int(dt['한국어'][i])
        dt['한국어'][i] = str(dt['한국어'][i])
    except:
        dt['한국어'][i] == dt['한국어'][i]



C:\Users\snrnq\AppData\Local\Temp/ipykernel_7720/4080634729.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = int(dt['한국어'][i])
C:\Users\snrnq\AppData\Local\Temp/ipykernel_7720/4080634729.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = str(dt['한국어'][i])


In [10]:
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dt['파일명']

0       KETI_SL_0000000001.avi
1       KETI_SL_0000000002.avi
2       KETI_SL_0000000003.avi
3       KETI_SL_0000000004.avi
4       KETI_SL_0000000005.avi
                 ...          
4699    KETI_SL_0000010371.avi
4700    KETI_SL_0000010372.avi
4701    KETI_SL_0000010373.avi
4702    KETI_SL_0000010374.avi
4703    KETI_SL_0000010375.avi
Name: 파일명, Length: 4704, dtype: object

In [33]:
#파일명 마지막 세글자.avi로 바꾸기 
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dt['파일명']

0       KETI_SL_0000000001.avi
1       KETI_SL_0000000002.avi
2       KETI_SL_0000000003.avi
3       KETI_SL_0000000004.avi
4       KETI_SL_0000000005.avi
                 ...          
4699    KETI_SL_0000010371.avi
4700    KETI_SL_0000010372.avi
4701    KETI_SL_0000010373.avi
4702    KETI_SL_0000010374.avi
4703    KETI_SL_0000010375.avi
Name: 파일명, Length: 4704, dtype: object

In [ ]:
import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']
dt1 = dt1[:3000]

In [34]:
## dt1 데이터 프레임 생성 
#전체 데이터 단어만 선택한 후 3000개 데이터 선택 

import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']
dt1 = dt1[:3000]

In [28]:
import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']  #단어만 골라내기
dt1 = dt1[:3000]   #0~2999
aa = dt1[['한국어']].copy()    #0~2999단어만 추출
list_a =aa['한국어'].value_counts()  #단어   횟수
list_a =list_a.reset_index()
list_a.columns = ['한국어','횟수']
#list_a = list_a.sort_values('한국어')
print(list_a['한국어'].sort_values())

TypeError: '<' not supported between instances of 'str' and 'int'

In [35]:
# 단어별로 동영상이 몇개 있는지 출력하는 list_a 데이터 프레임 생성 

#한국어컬럼만 추출
aa = dt1[['한국어']].copy()

#한국어 컬럼에 대한 개수 추출
list_a =aa['한국어'].value_counts()

#인덱스를 없애서 총 2개의 컬럼을 가진 데이터 프레임 생성 [한국어.횟수]
list_a =list_a.reset_index()

# 데이터프레임 컬렴명 변겅 
list_a.columns = ['한국어','횟수']
list_a = list_a.sort_values('한국어')
list_a.head()

#len(list_a)
#419

,한국어,횟수
0,0,8
22,1,8
49,10,8
328,100,7
325,1000,7


## numpy형태로 변환한 파일 담을 폴더 생성 

<폴더 구조> <br>
train 폴더 <br>
단어의 뜻을 이름으로 하는 폴더 419개  <br>
각각의 단어 폴더 안에 단어 당 촬영한 동영상 개수 만큼 폴더 생성   <br>
각각의 동영상 개수 폴더 안에 프레임 횟수만큼 폴더를 만들고 하나의 폴더안에 프레임 하나의 넘파이 배열이 담긴 파이썬 파일 첨부 

In [21]:
# 단어별 -동영상개수만큼의 폴더 생성하기 
import os
# train 폴더 생성
os.mkdir( 'c://Users//User//sign//train')

# 단어별 폴더 생성 
for k in range(419): 
    a = 'C://Users//User//sign//train//' + str(list_a['한국어'][k])  # 동영상 이름 
    os.mkdir(a)
    # 단어 폴더 안에 동영상의 횟수만큼 폴더 생성 
    for i in range(list_a['횟수'][k]):
        b= 'C://Users//User//sign//train//' + str(list_a['한국어'][k]) + '//' + str(i)
        os.mkdir(b)

In [ ]:
# 단어별 -동영상개수만큼의 폴더 생성하기 
import os
# train 폴더 생성
os.mkdir( 'e://Users//User//sign//train')

# 단어별 폴더 생성 
for k in range(419): 
    a = 'e://Users//User//sign//train//' + str(list_a['한국어'][k])  # 동영상 이름 
    os.mkdir(a)
    # 단어 폴더 안에 동영상의 횟수만큼 폴더 생성 
    for i in range(list_a['횟수'][k]):
        b= 'e://Users//User//sign//train//' + str(list_a['한국어'][k]) + '//' + str(i)
        os.mkdir(b)

## video3000에 있는 동영상 읽어와서 numpy배열로 변환한 후 위에서 만든 train 폴더에 저장 


In [26]:
#폴더의 개수 출력하는 함수 생성 
def dir_empty(dir_path):
    return len(os.listdir(dir_path))



# holistic 모델 사용 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # 3000개의 동영상 분석 
    for k in range(1000, 3001):
        # dt1['파일명']의 동영상 3000개 분석 
        #dt1 -> 단어별로 정리한 동영상 파일 3000개 
        
        videos = "C://Users//User//sign//video3000//" + dt1['파일명'][k] 

        
        #openCV를 열고 동영상을 읽어들인다. -> 동영상의 처음부터 끝까지 frame형태로 기록된다. 
        # -> 출력물 ret. frame 
        
        # ret이란 frame이 존재하는지 존재하지 않는지 True, False로 출력한다. 
        # 동영상이 끝나 더이상의 frame이 존재하지 않으면 False출력 
        
        cam = cv2.VideoCapture(videos)
        
        #출력하는 동영상의 해당 단어 kor에 담기 
        kor = dt1['한국어'][k]
        
        # videocount -> 해당 단어의 몇번째 동영상인지 출력 
        videocount =0

      # 
        while (True):
            #만약 해당 단어 폴더의 vidoecount의 이름을 가진 폴더의 길이가 0이라면(비어있다면)
            #  해당단어 - video count - frame 개수 별로 만든 폴더에 넘파이 파일 저장 
            if dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0:
                
                #프레임 개수를 알려주는 currentframe 
                currentframe = 0

                while (True):
                    # 여기서의 True 가 밑의 ret == True로 연결
                    #ret 이 True 라면 계속 반복, 즉 프레임이 남아있다면 계속해서 읽어들인다. 



                    # 위에서 읽은 cam을 frame별로 하나씩 읽어들이겠다.
                    ret,frame = cam.read()


                    # 만약 프레임이 남아있다면 
                    if ret == True:
                        
                        image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
                        keypoints = extract_keypoints(results) #result를 하나의 넘파이 배열로 만든다
                        
                        # 프레임 번호를 폴더명으로 하는 폴더 생성 
                        os.mkdir('C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe))
                        
                        #위에서 만든 폴더에 넘파이 배열 파일을 넣겠다. 
                        npy_path = 'C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe)+'//' +str(currentframe)
                        np.save(npy_path, keypoints)
                        
                        #다음프레임으로 이동 
                        currentframe +=1
                        


                    #만약 프레임이 끝났다면 즉, 하나의 동영상이 완료되었다면 종료 (두번째 while 문 종료)
                    else:
                        break
                
                # 하나의 동영상 추출 완료 ( if문 종료 )
                # dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0: 종료 
                # 완료를 알려주는 문장 출력 
                print(kor + '단어' + str(currentframe) + '프레임 추출 완료======================================')
                
                break
                
    #첫번째 while 문 
    #만약 ir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0 이 아니라면, 
    # 즉 단어 - 동영상 횟수를 나타내는 폴더가 비어있지 않다면 다음폴더로 이동 
    #ex ) 사자- 1번째 동영상의 폴더의 길이가 0이아니라면 사자-2 번째 폴더로 이동하고 위의 문장을 다시 실행 
    
            else:
                videocount +=1
                continue
        
        

낫단어103프레임 추출 완료======================================
낯선남자단어72프레임 추출 완료======================================
낯선사람단어80프레임 추출 완료======================================
낯선여자단어84프레임 추출 완료======================================
내년단어69프레임 추출 완료======================================
내일단어136프레임 추출 완료======================================
냄새나다단어136프레임 추출 완료======================================
노원구단어74프레임 추출 완료======================================
논단어86프레임 추출 완료======================================
농약단어99프레임 추출 완료======================================
누나단어136프레임 추출 완료======================================
누수단어92프레임 추출 완료======================================
누전단어81프레임 추출 완료======================================
누출단어88프레임 추출 완료======================================
눈단어78프레임 추출 완료======================================
다리단어72프레임 추출 완료======================================
다음단어136프레임 추출 완료======================================
달(월)단어151프레임 추출 완료======================================
대문앞단어107프레임 추출

옆쪽단어131프레임 추출 완료======================================
오늘단어58프레임 추출 완료======================================
오른쪽단어69프레임 추출 완료======================================
오른쪽-귀단어121프레임 추출 완료======================================
오른쪽-눈단어132프레임 추출 완료======================================
오빠단어120프레임 추출 완료======================================
옥상단어84프레임 추출 완료======================================
올해단어71프레임 추출 완료======================================
왼쪽단어73프레임 추출 완료======================================
왼쪽-귀단어121프레임 추출 완료======================================
왼쪽-눈단어121프레임 추출 완료======================================
욕실단어80프레임 추출 완료======================================
용산구단어85프레임 추출 완료======================================
우리집단어68프레임 추출 완료======================================
운동장단어81프레임 추출 완료======================================
월요일단어121프레임 추출 완료======================================
위단어107프레임 추출 완료======================================
위에단어71프레임 추출 완료======================================
위협단어78

42단어71프레임 추출 완료======================================
43단어128프레임 추출 완료======================================
44단어120프레임 추출 완료======================================
45단어136프레임 추출 완료======================================
46단어123프레임 추출 완료======================================
47단어128프레임 추출 완료======================================
48단어78프레임 추출 완료======================================
49단어113프레임 추출 완료======================================
50단어119프레임 추출 완료======================================
51단어128프레임 추출 완료======================================
52단어187프레임 추출 완료======================================
53단어128프레임 추출 완료======================================
54단어71프레임 추출 완료======================================
55단어115프레임 추출 완료======================================
56단어117프레임 추출 완료======================================
57단어137프레임 추출 완료======================================
58단어137프레임 추출 완료======================================
59단어126프레임 추출 완료======================================
60단어80프레임 추출 

뜨거운물단어81프레임 추출 완료======================================
마당단어111프레임 추출 완료======================================
마포구단어88프레임 추출 완료======================================
말려주세요단어88프레임 추출 완료======================================
말벌단어115프레임 추출 완료======================================
맹견단어105프레임 추출 완료======================================
머리단어87프레임 추출 완료======================================
멧돼지단어92프레임 추출 완료======================================
목단어121프레임 추출 완료======================================
목요일단어72프레임 추출 완료======================================
무너지다단어60프레임 추출 완료======================================
무릎단어70프레임 추출 완료======================================
문틈단어113프레임 추출 완료======================================
물단어141프레임 추출 완료======================================
밑에단어67프레임 추출 완료======================================
바다단어91프레임 추출 완료======================================
반점생기다단어116프레임 추출 완료======================================
발단어56프레임 추출 완료======================================
발가락단어110프레임

작년단어82프레임 추출 완료======================================
작은방단어91프레임 추출 완료======================================
장난감단어79프레임 추출 완료======================================
장단지단어96프레임 추출 완료======================================
절단단어77프레임 추출 완료======================================
제초제단어116프레임 추출 완료======================================
조난단어87프레임 추출 완료======================================
종로구단어79프레임 추출 완료======================================
주단어136프레임 추출 완료======================================
중구단어86프레임 추출 완료======================================
중랑구단어84프레임 추출 완료======================================
지난단어170프레임 추출 완료======================================
지혈대단어130프레임 추출 완료======================================
진통제단어116프레임 추출 완료======================================
질식단어86프레임 추출 완료======================================
집단어64프레임 추출 완료======================================
집단폭행단어111프레임 추출 완료======================================
차밖단어90프레임 추출 완료======================================
차안단어92프레임 추출 완

...

협박단어208프레임 추출 완료======================================
형단어166프레임 추출 완료======================================
호흡곤란단어196프레임 추출 완료======================================
호흡기단어115프레임 추출 완료======================================
홍수단어126프레임 추출 완료======================================
화상단어114프레임 추출 완료======================================
화약단어107프레임 추출 완료======================================
화요일단어186프레임 추출 완료======================================
화장실단어201프레임 추출 완료======================================
화재단어110프레임 추출 완료======================================
장단지단어127프레임 추출 완료======================================
0단어197프레임 추출 완료======================================
1단어171프레임 추출 완료======================================
2단어171프레임 추출 완료======================================
3단어149프레임 추출 완료======================================
4단어191프레임 추출 완료======================================
5단어178프레임 추출 완료======================================
6단어167프레임 추출 완료======================================
7단어156프레임 추출

공장단어166프레임 추출 완료======================================
관악구단어153프레임 추출 완료======================================
광진구단어161프레임 추출 완료======================================
교통사고단어182프레임 추출 완료======================================
구급대단어110프레임 추출 완료======================================
구급대원단어185프레임 추출 완료======================================
구급차단어97프레임 추출 완료======================================
구로구단어166프레임 추출 완료======================================
구청단어70프레임 추출 완료======================================
구해주세요단어167프레임 추출 완료======================================
귀단어63프레임 추출 완료======================================
금가다단어61프레임 추출 완료======================================
금요일단어196프레임 추출 완료======================================
금천구단어82프레임 추출 완료======================================
급류단어95프레임 추출 완료======================================
기절단어71프레임 추출 완료======================================
기절하다단어71프레임 추출 완료======================================
깔리다단어66프레임 추출 완료======================================
끓는

약국단어66프레임 추출 완료======================================
약사단어68프레임 추출 완료======================================
양천구단어94프레임 추출 완료======================================
어깨단어61프레임 추출 완료======================================
어제단어53프레임 추출 완료======================================
어지러움단어66프레임 추출 완료======================================
언니단어50프레임 추출 완료======================================
얼굴단어66프레임 추출 완료======================================
엄마단어60프레임 추출 완료======================================
엘리베이터단어66프레임 추출 완료======================================
여자친구단어82프레임 추출 완료======================================
여학생단어181프레임 추출 완료======================================
연기단어77프레임 추출 완료======================================
연락해주세요단어89프레임 추출 완료======================================
열단어53프레임 추출 완료======================================
열나다단어69프레임 추출 완료======================================
열어주세요단어86프레임 추출 완료======================================
엽총단어158프레임 추출 완료======================================
영등포구단어81프레

19단어174프레임 추출 완료======================================
20단어77프레임 추출 완료======================================
21단어143프레임 추출 완료======================================
22단어154프레임 추출 완료======================================
23단어193프레임 추출 완료======================================
24단어197프레임 추출 완료======================================
25단어192프레임 추출 완료======================================
26단어198프레임 추출 완료======================================
27단어88프레임 추출 완료======================================
28단어77프레임 추출 완료======================================
29단어202프레임 추출 완료======================================
30단어174프레임 추출 완료======================================
31단어81프레임 추출 완료======================================
32단어207프레임 추출 완료======================================
33단어86프레임 추출 완료======================================
34단어191프레임 추출 완료======================================
35단어179프레임 추출 완료======================================
36단어194프레임 추출 완료======================================
37단어197프레임 추출 완

KeyError: 3000

In [ ]:
## 3차원 형태의 넘파이 배열을 모델에 넣기 위해서는 모든 프레임의 개수가 같아야 한다.

# 프레임의 개수가 가장 많은 영상을 찾아 그 영상에 맞춰 모든 영상의 마지막 프레임을 해당 프레임의 개수만큼 복사한다. 



In [45]:
list_a

#len(list_a)   #419

,한국어,횟수
0,0,8
22,1,8
49,10,8
328,100,7
325,1000,7
...,...,...
189,화상,7
188,화약,7
187,화요일,7
186,화장실,7


In [62]:
train_path=[]

for i in range(len(list_a)):

    videos2 = "C://Users//User//sign//train//" + str(list_a['한국어'][i])
    train_path.append(videos2)
    
# len(train_path)   #419

train_path

['C://Users//User//sign//train//0',
 'C://Users//User//sign//train//51',
 'C://Users//User//sign//train//58',
 'C://Users//User//sign//train//57',
 'C://Users//User//sign//train//56',
 'C://Users//User//sign//train//55',
 'C://Users//User//sign//train//54',
 'C://Users//User//sign//train//53',
 'C://Users//User//sign//train//52',
 'C://Users//User//sign//train//50',
 'C://Users//User//sign//train//61',
 'C://Users//User//sign//train//49',
 'C://Users//User//sign//train//48',
 'C://Users//User//sign//train//47',
 'C://Users//User//sign//train//46',
 'C://Users//User//sign//train//44',
 'C://Users//User//sign//train//43',
 'C://Users//User//sign//train//42',
 'C://Users//User//sign//train//60',
 'C://Users//User//sign//train//62',
 'C://Users//User//sign//train//40',
 'C://Users//User//sign//train//72',
 'C://Users//User//sign//train//1',
 'C://Users//User//sign//train//78',
 'C://Users//User//sign//train//77',
 'C://Users//User//sign//train//76',
 'C://Users//User//sign//train//75',
 'C

In [ ]:
train_path=[]

for i in range(len(list_a)):

    videos2 = "C://Users//User//sign//train//" + str(list_a['한국어'][i])
    train_path.append(videos2)
    
# len(train_path)   #419

train_path

In [80]:
import os

train_path_count=[]

for k in range(len(train_path)):

    file_list = os.listdir(train_path[k])
    
    for p in range(len(file_list)):
        train_path_count.append(train_path[k]+ '//' + file_list[p] )
    
    
path = train_path_count

len_file_list=[]
path_last=[]

for j in range(len(path)):
    file_list = os.listdir(path[j])
    print(path[j], len(file_list))
    len_file_list.append(len(file_list))
    path_last.append(path[j])


C://Users//User//sign//train//0//0 141
C://Users//User//sign//train//0//1 145
C://Users//User//sign//train//0//2 85
C://Users//User//sign//train//0//3 120
C://Users//User//sign//train//0//4 191
C://Users//User//sign//train//0//5 87
C://Users//User//sign//train//0//6 197
C://Users//User//sign//train//0//7 203
C://Users//User//sign//train//51//0 150
C://Users//User//sign//train//51//1 72
C://Users//User//sign//train//51//2 131
C://Users//User//sign//train//51//3 128
C://Users//User//sign//train//51//4 174
C://Users//User//sign//train//51//5 100
C://Users//User//sign//train//51//6 169
C://Users//User//sign//train//51//7 192
C://Users//User//sign//train//58//0 154
C://Users//User//sign//train//58//1 75
C://Users//User//sign//train//58//2 131
C://Users//User//sign//train//58//3 137
C://Users//User//sign//train//58//4 100
C://Users//User//sign//train//58//5 94
C://Users//User//sign//train//58//6 145
C://Users//User//sign//train//58//7 185
C://Users//User//sign//train//57//0 161
C://Users//Us

C://Users//User//sign//train//6//6 167
C://Users//User//sign//train//6//7 185
C://Users//User//sign//train//5//0 151
C://Users//User//sign//train//5//1 62
C://Users//User//sign//train//5//2 88
C://Users//User//sign//train//5//3 120
C://Users//User//sign//train//5//4 131
C://Users//User//sign//train//5//5 184
C://Users//User//sign//train//5//6 178
C://Users//User//sign//train//5//7 180
C://Users//User//sign//train//4//0 144
C://Users//User//sign//train//4//1 136
C://Users//User//sign//train//4//2 86
C://Users//User//sign//train//4//3 132
C://Users//User//sign//train//4//4 161
C://Users//User//sign//train//4//5 159
C://Users//User//sign//train//4//6 191
C://Users//User//sign//train//4//7 176
C://Users//User//sign//train//3//0 146
C://Users//User//sign//train//3//1 148
C://Users//User//sign//train//3//2 101
C://Users//User//sign//train//3//3 147
C://Users//User//sign//train//3//4 190
C://Users//User//sign//train//3//5 185
C://Users//User//sign//train//3//6 149
C://Users//User//sign//train

C://Users//User//sign//train//손//5 136
C://Users//User//sign//train//손//6 61
C://Users//User//sign//train//소화전//0 106
C://Users//User//sign//train//소화전//1 112
C://Users//User//sign//train//소화전//2 121
C://Users//User//sign//train//소화전//3 127
C://Users//User//sign//train//소화전//4 138
C://Users//User//sign//train//소화전//5 153
C://Users//User//sign//train//소화전//6 131
C://Users//User//sign//train//소화기//0 95
C://Users//User//sign//train//소화기//1 100
C://Users//User//sign//train//소화기//2 106
C://Users//User//sign//train//소화기//3 105
C://Users//User//sign//train//소화기//4 125
C://Users//User//sign//train//소화기//5 137
C://Users//User//sign//train//소화기//6 96
C://Users//User//sign//train//소방차//0 98
C://Users//User//sign//train//소방차//1 99
C://Users//User//sign//train//소방차//2 108
C://Users//User//sign//train//소방차//3 119
C://Users//User//sign//train//소방차//4 120
C://Users//User//sign//train//소방차//5 144
C://Users//User//sign//train//소방차//6 119
C://Users//User//sign//train//소방관//0 93
C://Users//User//sign//tra

C://Users//User//sign//train//허리//3 85
C://Users//User//sign//train//허리//4 94
C://Users//User//sign//train//허리//5 199
C://Users//User//sign//train//허리//6 71
C://Users//User//sign//train//해열제//0 113
C://Users//User//sign//train//해열제//1 110
C://Users//User//sign//train//해열제//2 116
C://Users//User//sign//train//해열제//3 124
C://Users//User//sign//train//해열제//4 121
C://Users//User//sign//train//해열제//5 144
C://Users//User//sign//train//해열제//6 121
C://Users//User//sign//train//해독제//0 99
C://Users//User//sign//train//해독제//1 100
C://Users//User//sign//train//해독제//2 102
C://Users//User//sign//train//해독제//3 110
C://Users//User//sign//train//해독제//4 114
C://Users//User//sign//train//해독제//5 118
C://Users//User//sign//train//해독제//6 103
C://Users//User//sign//train//해(연)//0 70
C://Users//User//sign//train//해(연)//1 81
C://Users//User//sign//train//해(연)//2 83
C://Users//User//sign//train//해(연)//3 73
C://Users//User//sign//train//해(연)//4 75
C://Users//User//sign//train//해(연)//5 103
C://Users//User//sign//

C://Users//User//sign//train//남학생//6 77
C://Users//User//sign//train//남편//0 65
C://Users//User//sign//train//남편//1 63
C://Users//User//sign//train//남편//2 147
C://Users//User//sign//train//남편//3 88
C://Users//User//sign//train//남편//4 92
C://Users//User//sign//train//남편//5 176
C://Users//User//sign//train//남편//6 73
C://Users//User//sign//train//남자친구//0 70
C://Users//User//sign//train//남자친구//1 81
C://Users//User//sign//train//남자친구//2 76
C://Users//User//sign//train//남자친구//3 83
C://Users//User//sign//train//남자친구//4 91
C://Users//User//sign//train//남자친구//5 156
C://Users//User//sign//train//남자친구//6 80
C://Users//User//sign//train//끓는물//0 71
C://Users//User//sign//train//끓는물//1 76
C://Users//User//sign//train//끓는물//2 84
C://Users//User//sign//train//끓는물//3 95
C://Users//User//sign//train//끓는물//4 112
C://Users//User//sign//train//끓는물//5 124
C://Users//User//sign//train//끓는물//6 94
C://Users//User//sign//train//깔리다//0 66
C://Users//User//sign//train//깔리다//1 175
C://Users//User//sign//train//깔리다/

C://Users//User//sign//train//바다//4 90
C://Users//User//sign//train//바다//5 98
C://Users//User//sign//train//바다//6 152
C://Users//User//sign//train//밑에//0 58
C://Users//User//sign//train//밑에//1 63
C://Users//User//sign//train//밑에//2 85
C://Users//User//sign//train//밑에//3 67
C://Users//User//sign//train//밑에//4 85
C://Users//User//sign//train//밑에//5 116
C://Users//User//sign//train//밑에//6 181
C://Users//User//sign//train//물//0 108
C://Users//User//sign//train//물//1 53
C://Users//User//sign//train//물//2 100
C://Users//User//sign//train//물//3 141
C://Users//User//sign//train//물//4 114
C://Users//User//sign//train//물//5 166
C://Users//User//sign//train//물//6 72
C://Users//User//sign//train//문틈//0 73
C://Users//User//sign//train//문틈//1 102
C://Users//User//sign//train//문틈//2 88
C://Users//User//sign//train//문틈//3 113
C://Users//User//sign//train//문틈//4 93
C://Users//User//sign//train//문틈//5 115
C://Users//User//sign//train//문틈//6 76
C://Users//User//sign//train//무릎//0 92
C://Users//User//sign

C://Users//User//sign//train//딸//0 131
C://Users//User//sign//train//딸//1 55
C://Users//User//sign//train//딸//2 155
C://Users//User//sign//train//딸//3 122
C://Users//User//sign//train//딸//4 121
C://Users//User//sign//train//딸//5 184
C://Users//User//sign//train//딸//6 60
C://Users//User//sign//train//동생//0 131
C://Users//User//sign//train//동생//1 59
C://Users//User//sign//train//동생//2 161
C://Users//User//sign//train//동생//3 122
C://Users//User//sign//train//동생//4 107
C://Users//User//sign//train//동생//5 161
C://Users//User//sign//train//동생//6 156
C://Users//User//sign//train//동대문구//0 77
C://Users//User//sign//train//동대문구//1 99
C://Users//User//sign//train//동대문구//2 80
C://Users//User//sign//train//동대문구//3 84
C://Users//User//sign//train//동대문구//4 110
C://Users//User//sign//train//동대문구//5 98
C://Users//User//sign//train//동대문구//6 92
C://Users//User//sign//train//독사//0 80
C://Users//User//sign//train//독사//1 81
C://Users//User//sign//train//독사//2 94
C://Users//User//sign//train//독사//3 98
C://Us

In [50]:
list_a[['한국어','횟수']][list_a['한국어']==0 &&(list_a['한국어']==9)]

SyntaxError: invalid syntax (Temp/ipykernel_11512/126096718.py, line 1)

In [30]:
train_path=[]

for i in range(len(list_a)):

    videos2 = "E://Users//User//sign//test//" + str(list_a['한국어'][i])
    train_path.append(videos2)
    
# len(train_path)   #419

train_path

['E://Users//User//sign//test//0',
 'E://Users//User//sign//test//51',
 'E://Users//User//sign//test//58',
 'E://Users//User//sign//test//57',
 'E://Users//User//sign//test//56',
 'E://Users//User//sign//test//55',
 'E://Users//User//sign//test//54',
 'E://Users//User//sign//test//53',
 'E://Users//User//sign//test//52',
 'E://Users//User//sign//test//50',
 'E://Users//User//sign//test//61',
 'E://Users//User//sign//test//49',
 'E://Users//User//sign//test//48',
 'E://Users//User//sign//test//47',
 'E://Users//User//sign//test//46',
 'E://Users//User//sign//test//44',
 'E://Users//User//sign//test//43',
 'E://Users//User//sign//test//42',
 'E://Users//User//sign//test//60',
 'E://Users//User//sign//test//62',
 'E://Users//User//sign//test//40',
 'E://Users//User//sign//test//72',
 'E://Users//User//sign//test//1',
 'E://Users//User//sign//test//78',
 'E://Users//User//sign//test//77',
 'E://Users//User//sign//test//76',
 'E://Users//User//sign//test//75',
 'E://Users//User//sign//test/

In [29]:
import os

train_path_count=[]

for k in range(len(train_path)):

    file_list = os.listdir(train_path[k])
    
    for p in range(len(file_list)):
        train_path_count.append(train_path[k]+ '//' + file_list[p] )
    
    
path = train_path_count

len_file_list=[]
path_last=[]

for j in range(len(path)):
    file_list = os.listdir(path[j])
    print(path[j], len(file_list))
    len_file_list.append(len(file_list))
    path_last.append(path[j])

NameError: name 'train_path' is not defined

In [76]:
#print(len_file_list)

max(len_file_list)   #옆쪽//6     0~431

432

In [ ]:
#디렉토리를 복사하는 법

import os
import shutil

#~0을 ~0_2 이름으로 복사함
shutil.copytree("C://Users//User//sign//test//0", "C://Users//User//sign//test//0_2")

for f in os.listdir("C://Users//User//sign//test//0_2"):
    print(f)

In [ ]:
#train 폴더로 했다가 잘못하면 위험하기 때문에
#test 폴더로 시도해볼 것..!

In [98]:
file_list_1=os.listdir("C://Users//User//sign//test")

print(file_list_1)
print(file_list_1[1])
print(len(file_list_1))

['0', '93', '수요일']
93
3


In [105]:
for i in range(len(file_list_1)):
    path="C://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    print(file_list_2)

['0', '1', '2', '3', '4', '5', '6', '7']
['0', '1', '2', '3', '4', '5', '6']
['0', '1', '2', '3', '4', '5', '6']


In [106]:
for i in range(len(file_list_1)):
    path="C://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        print(path2)

C://Users//User//sign//test//0//0
C://Users//User//sign//test//0//1
C://Users//User//sign//test//0//2
C://Users//User//sign//test//0//3
C://Users//User//sign//test//0//4
C://Users//User//sign//test//0//5
C://Users//User//sign//test//0//6
C://Users//User//sign//test//0//7
C://Users//User//sign//test//93//0
C://Users//User//sign//test//93//1
C://Users//User//sign//test//93//2
C://Users//User//sign//test//93//3
C://Users//User//sign//test//93//4
C://Users//User//sign//test//93//5
C://Users//User//sign//test//93//6
C://Users//User//sign//test//수요일//0
C://Users//User//sign//test//수요일//1
C://Users//User//sign//test//수요일//2
C://Users//User//sign//test//수요일//3
C://Users//User//sign//test//수요일//4
C://Users//User//sign//test//수요일//5
C://Users//User//sign//test//수요일//6


In [89]:
for i in range(len(file_list_1)):
    path="E://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        file_list_3=os.listdir(path2)
        print(path2, len(file_list_3))

E://Users//User//sign//test//0//0 141
E://Users//User//sign//test//0//1 145
E://Users//User//sign//test//0//2 85
E://Users//User//sign//test//0//3 120
E://Users//User//sign//test//0//4 191
E://Users//User//sign//test//0//5 87
E://Users//User//sign//test//0//6 197
E://Users//User//sign//test//0//7 203
E://Users//User//sign//test//93//0 144
E://Users//User//sign//test//93//1 76
E://Users//User//sign//test//93//2 166
E://Users//User//sign//test//93//3 120
E://Users//User//sign//test//93//4 191
E://Users//User//sign//test//93//5 94
E://Users//User//sign//test//93//6 61
E://Users//User//sign//test//수요일//0 116
E://Users//User//sign//test//수요일//1 50
E://Users//User//sign//test//수요일//2 121
E://Users//User//sign//test//수요일//3 87
E://Users//User//sign//test//수요일//4 126
E://Users//User//sign//test//수요일//5 189
E://Users//User//sign//test//수요일//6 58


In [90]:
for i in range(len(file_list_1)):
    path="E://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        file_list_3=os.listdir(path2)
        print(path2, 203-len(file_list_3))

E://Users//User//sign//test//0//0 62
E://Users//User//sign//test//0//1 58
E://Users//User//sign//test//0//2 118
E://Users//User//sign//test//0//3 83
E://Users//User//sign//test//0//4 12
E://Users//User//sign//test//0//5 116
E://Users//User//sign//test//0//6 6
E://Users//User//sign//test//0//7 0
E://Users//User//sign//test//93//0 59
E://Users//User//sign//test//93//1 127
E://Users//User//sign//test//93//2 37
E://Users//User//sign//test//93//3 83
E://Users//User//sign//test//93//4 12
E://Users//User//sign//test//93//5 109
E://Users//User//sign//test//93//6 142
E://Users//User//sign//test//수요일//0 87
E://Users//User//sign//test//수요일//1 153
E://Users//User//sign//test//수요일//2 82
E://Users//User//sign//test//수요일//3 116
E://Users//User//sign//test//수요일//4 77
E://Users//User//sign//test//수요일//5 14
E://Users//User//sign//test//수요일//6 145


In [ ]:
import os
import shutil
 
def customized_copy(input_path, output_path,file_name):
    subdir_list = os.listdir(input_path)
    for subdir in subdir_list:
        print(subdir)
        raw_file_path = os.path.join(input_path, subdir, file_name)#type：file_name_str
        target_file_path = os.path.join(output_path, subdir, file_name)#type：file_name_str
        shutil.copyfile(raw_file_path,target_file_path)
 
 if __name__ == '__main__':
    input_path = "/data/dataset/xxx/06_results_rai_30_0415/"
    output_path = "/data/dataset/xxx/06_results/"
    file_name = "seg_kidney_tumor.nii.gz"
    customized_copy(input_path, output_path,file_name)

In [92]:
import os
import shutil
def customized_copy(input_path):
    sbbdir_list=os.listdir(input_path) 
    for i in range(len(file_list_1)):
        path="E://Users//User//sign//test//" + str(file_list_1[i])
        file_list_2=os.listdir(path)
        for p in range(len(file_list_2)):
            path2=path+'//'+str(file_list_2[p])
            file_list_3=os.listdir(path2)
            input_path=path2[len(file_list_3)]
    
print(customized_copy(input_path))

NameError: name 'input_path' is not defined

In [ ]:
import os 
import shutil
 
data_path = ''
 
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith("npy"): #확장자 
            old_file_path = os.path.join(root, file)
            #print(old_file_path)
            new_path = ''
            if not os.path.exists(new_path):  # make new dir
                os.makedirs(new_path)
            new_file_path = new_path + '/' + file
            print(new_file_path)
            shutil.copyfile(old_file_path, new_file_path) # copy 
print('finished!')

In [106]:
import os
import shutil
count_list3=[]
for i in range(len(file_list_1)):
    path="E://Users//User//sign//test//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        file_list_3=os.listdir(path2)
        count_list3.append(len(file_list_3))
        for j in range(len(file_list_3)):
            path3=path2+'//'+str(file_list_3[j])
            #print(path3)        
        while len(file_list_3) < 203:
            shutil.copytree(path2+'//'+str(max(count_list3)-1), path2+'//')             
#print(max(count_list3))
#len(file_list_3)
           # print(path3)

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'E://Users//User//sign//test//0//0//140'

In [101]:
import os
import shutil
count_list3=[]
cost=[]
file_list_1=os.listdir("E://Users//User//sign//test")
for i in range(len(file_list_1)):
    path="E:///Users//User//sign//test//"+str(file_list_1[i])
    file_list_2=os.listdir(path)
    for j in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[j])
        file_list_3=os.listdir(path2)
        for k in path2:
            if len(file_list_3) <203:
                cost.append(203-len(file_list_3))
print(cost)

[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 59, 59, 59, 5

In [97]:
#맨 끝 폴더(디렉토리)명만 얻는 법

os.path.basename('C://Users//User//sign//test//'+str(len(file_list_3)))

'141'

In [ ]:
import  os,time,shutil,datetime
def rmdir(deldir,N):
    dellist=os.listdir(deldir)
    deldate= datetime.timedelta(days=N)
    now = datetime.datetime.now()
    for  i  in  dellist:
        os.chdir(deldir)
        ctime = datetime.datetime.fromtimestamp(os.path.getctime(i))
        if ctime < (now - deldate):
            #print ("%s rm  dir :%s" %(now,i))
            if os.path.isfile(i):
                os.remove(i)
            else:
                shutil.rmtree(i)
 
#Timed C--logs deletion
#Delete the directory at midnight every day and keep the last few days
rmdir('C:\logs\Adf.NumberServer',7)
rmdir('E:\',7)
 

In [ ]:
import os
import shutil
 
    # 删除文件夹下的文件&&保留但清空子文件夹
def del_file(filepath):
    print("hello")
    listdir = os.listdir(filepath)  # 获取文件和子文件夹
    print(listdir)
    for dirname in listdir:
        dirname = filepath + "//" + dirname
        if os.path.isfile(dirname): # 是文件
            print(dirname)
            os.remove(dirname)      # 删除文件
        elif os.path.isdir(dirname):        # 是子文件夹
            print(dirname)
            dellist = os.listdir(dirname)
            for f in dellist:               # 遍历该子文件夹
                file_path = os.path.join(dirname, f)
                if os.path.isfile(file_path):       # 删除子文件夹下文件
                    os.remove(file_path)
                elif os.path.isdir(file_path):      # 强制删除子文件夹下的子文件夹
                    shutil.rmtree(file_path)
     

In [128]:
import os
import shutil
#filename=[]
listdir=os.listdir('E://Users//User//sign//test')
for i in listdir:
    listdir2=os.listdir('E://Users//User//sign//test'+'//'+str(i))
    for j in range(len(listdir2)):
        listdir3=os.listdir('E://Users//User//sign//test'+'//'+str(i)+'//'+str(j))
        if os.path.isdir(listdir3):
            dellist=os.listdir(listdir3)
            print(dellist)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not list

# 6. 데이터 라벨링하기 


### 동영상 데이터 라벨링하기


In [ ]:
# 다른 액션에 대한 레이블 만들기 
# 필요한 데이터 임포트
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
# 정답데이터 라벨링 하기 
label_map = {label:num for num, label in enumerate(dt1['한국어'])} # enumerate-> 0 부터 순서대로 나타내 주는 것 

In [ ]:
label_map

In [ ]:
##### 프레임 개수만 맞추면 됩니다. 

In [ ]:
# 파일 불러와서 해당 영상에 대한 정답 데이터 labels에 담기 
# labeling actions to labels 

sequences, labels = [], []
DATA_PATH  =  'C://Users//User//sign//train//'
for action in dt1['한국어']:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int): #단어 안에 있는 폴더의 이름을 리스트로 출력하고 int형으로 변경 
        window = []
        for currentframe in range(****sequence_length(프레임 개수****)):
            # 데이터 파이썬 파일로 불러오기 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(currentfrmae)))
            # 리스트에 append 하기 
            window.append(res)
            #sequence라는 리스트안에 window가 리스트 형터로 존재 
            #리스트 안의 리스트 
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape
# 총 90개의 비디오, 1편 당 30시퀀스 /  # (1662,)키포인트 -> 468*3 + 33*4 + 21*3 + 21*3 각(xyz 즉 세개씩 곱한다.)


In [ ]:
np.array(labels).shape  #(90,) 라벨링한 정답 데이터 

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape
#90 channels, 30 frames, 1662 keypoints (68*3 + 33*4 + 21*3 + 21*3)

In [ ]:
# 원핫인코딩 하기 
# one hot encoding 
y = to_categorical(labels).astype(int)

In [ ]:
# 데이터 나누기
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # 5%가 테스트 되겠다

In [ ]:
y_test.shape

# LSTM Neural Network 만들기 

Build and Train LSTM Neural Network

In [ ]:
# 모델 생성 -> cnn 넣었을 때, 넣지 않았을 때 모두 해보기 

In [ ]:
# 모델 임포트 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense  #LSTM layer 를  쓰겠다.  
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# set call back ( prevent over fitting )
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# 이 타입 쓰는 이유 CNN 과 비교 시 장점 
# 미디아 type + LSTM 레이어
# 1. Less DaTa Required 
# 2. Faster To train
# 3. Faster Detections 

In [ ]:
model = Sequential()  # 시작 
# 30, 1662) 각각의 비디오 30 시퀀스 1662 키포인트


#######
# cnn 모델 추가 

########


model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) #64 유닛 -> LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))# 다음은 dense슬라이드라 sequences 필요없어
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

In [ ]:
# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

 -  결과보기 
 - visualize the result of model



 cmd 창을 킨다 <br>
open cmd  <br>
<br>
C:\Users\User>cd Logs <br>
<br>
C:\Users\User\Logs>cd Train<br>
<br>
C:\Users\User\Logs\train>dir<br>

- 파일을 텐서플로우로 실행시킨다. (run the file )
C:\Users\User\Logs\train>tensorboard --logdir=. 

- 링크를 복사하여 크롬창에 띄운다 ( copy the like and open it at Chrome)

# 예측하기 
8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 모델 저장 
9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')

# 모델 평가 
10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)
# 이원교차표 출력 

In [ ]:
accuracy_score(ytrue, yhat)

# 실시간 예측하기 
11. Test in Real Time

In [5]:
# 동영상에 점 찍기 

def draw_landmarks(image, results):  # 동영상의 frame과 동영상에서 받은 result를 받는다. 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # 얼굴의 점을 연결하여 그린다.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # POSE를 연결햐여 그린다.
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 왼손 그리기
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 오른손 그리기 

In [6]:
# 더 정확한 판단을 위해 각각의 노드에 두개의 점을 추가한다. 
#add aditional 2 spots to the image

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables #30개 차면 detect 할 수 있게
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] #마지막 30프레임
        
        if len(sequence) == 30: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()